In [21]:
import json
import numpy as np
import os
from scipy.linalg import expm, logm
import sys

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Statevector, Operator, SparsePauliOp

In [22]:
def inner_product(vec1, vec2):
    """Compute the inner product of two complex vectors."""
    return np.vdot(vec1, vec2)  # vdot handles complex conjugation automatically

In [23]:
# Define the file to store embeddings
EMBEDDINGS_FILE = "embeddings.json"

def load_embeddings():
    """Loads stored embeddings from a JSON file if it exists."""
    if os.path.exists(EMBEDDINGS_FILE):
        with open(EMBEDDINGS_FILE, "r") as f:
            return json.load(f)
    return {}

def get_embeddings(word, stored_embeddings):
    """Retrieves embedding from cache or fetches it from OpenAI if not cached."""
    if word in stored_embeddings:
        print(f"Loaded cached embedding for '{word}'")
        return stored_embeddings[word]

def encode_to_qubit(arr):
    """Encodes a real array of any even size into a qubit state using amplitude encoding."""
    assert len(arr) % 2 == 0, "Array length must be even"
    complex_vec = np.array([arr[i] + 1j * arr[i+1] for i in range(0, len(arr), 2)])
    norm = np.linalg.norm(complex_vec)
    return complex_vec / norm if norm != 0 else complex_vec

words = ["cat", "dog", "car"]

# Load existing embeddings
stored_embeddings = load_embeddings()

# Retrieve embeddings (from file or OpenAI)
embeddings = {word: get_embeddings(word, stored_embeddings) for word in words}

# Extract individual vectors correctly
cat_vec = embeddings["cat"]
dog_vec = embeddings["dog"]
car_vec = embeddings["car"]

# Encode them into qubit states
cat_state = encode_to_qubit(cat_vec)
dog_state = encode_to_qubit(dog_vec)
car_state = encode_to_qubit(car_vec)

# Compute the inner product
cat_dog_inner_prod = inner_product(cat_state, dog_state)
cat_car_inner_prod = inner_product(cat_state, car_state)
dog_car_inner_prod = inner_product(dog_state, car_state)

# Compute the magnitude of the inner product
cat_dog_magnitude = np.abs(cat_dog_inner_prod)
cat_car_magnitude = np.abs(cat_car_inner_prod)
dog_car_magnitude = np.abs(dog_car_inner_prod)

# Display the results
print("cat/dog Inner product:", cat_dog_inner_prod)
print("cat/car Inner product:", cat_car_inner_prod)
print("dog/car Inner product:", dog_car_inner_prod)

print("cat/dog Magnitude of inner product:", cat_dog_magnitude)
print("cat/car Magnitude of inner product:", cat_car_magnitude)
print("dog/car Magnitude of inner product:", dog_car_magnitude)


Loaded cached embedding for 'cat'
Loaded cached embedding for 'dog'
Loaded cached embedding for 'car'
cat/dog Inner product: (0.862957645542653-0.016694274937911695j)
cat/car Inner product: (0.845207835908355-0.010998340229826991j)
dog/car Inner product: (0.8330066666719805+0.00492076801033331j)
cat/dog Magnitude of inner product: 0.8631191092869059
cat/car Magnitude of inner product: 0.8452793913072149
dog/car Magnitude of inner product: 0.8330212006172325


In [24]:
def normalize(vector):
    """Normalize a complex vector."""
    norm = np.linalg.norm(vector)
    if norm == 0:
        raise ValueError("Zero vector cannot be normalized.")
    return vector / norm

def pair_real_to_complex(vector):
    """Convert a real-valued vector into a complex-valued vector by pairing elements."""
    if len(vector) % 2 != 0:
        raise ValueError("Vector length must be even to pair into complex numbers.")
    complex_vector = np.array([vector[i] + 1j * vector[i+1] for i in range(0, len(vector), 2)])
    return complex_vector

def pad_vector(vector, target_size):
    """Pad the vector with zeros to the nearest power of 2."""
    if len(vector) > target_size:
        raise ValueError("Vector is larger than the target size.")
    padded_vector = np.zeros(target_size, dtype=complex)
    padded_vector[:len(vector)] = vector  # Copy original values
    return padded_vector

def get_rotation_angles(state):
    """Compute rotation angles theta and phi from state vector."""
    norm_left = np.linalg.norm(state[:len(state)//2])
    norm_right = np.linalg.norm(state[len(state)//2:])
    
    theta = 2 * np.arctan2(norm_right, norm_left)  # Ry rotation
    phi = np.angle(state[len(state)//2]) if (norm_left > 0 and norm_right > 0) else 0.0  # Rz phase shift
    
    return theta, phi

def apply_recursive_state_preparation(qc, qubits, state, control=None):
    n = len(state)
    if n == 1:
        return

    half = n // 2
    theta, phi = get_rotation_angles(state)
    target = qubits[0]

    # If there's no control, apply the rotation directly
    if control is None:
        qc.rz(phi, target)
        qc.ry(theta, target)
    else:
        # Apply controlled rotations (these gates are not built-in; one must construct them)
        qc.crz(phi, control, target)
        qc.cry(theta, control, target)

    # For each branch, the next set of qubits will be used.
    # Here you would prepare left and right branches conditionally.
    # This is a nontrivial step and typically involves multi-controlled rotations.
    left_state = normalize(state[:half]) if np.linalg.norm(state[:half]) > 0 else np.zeros(half, dtype=complex)
    right_state = normalize(state[half:]) if np.linalg.norm(state[half:]) > 0 else np.zeros(half, dtype=complex)

    # Recurse: use qubits[1:] for the sub-circuits, with target qubit as the new control.
    if len(qubits) > 1:
        apply_recursive_state_preparation(qc, qubits[1:], left_state, control=target)
        apply_recursive_state_preparation(qc, qubits[1:], right_state, control=target)


def prepare_amplitude_encoding(vector, state_vec=True):
    """Create a Qiskit quantum circuit that prepares the amplitude-encoded state."""
    vector = np.array(vector)
    
    # Step 1: Convert reals to complex numbers
    complex_vector = pair_real_to_complex(vector)
    
    # Step 2: Compute the nearest power of 2
    num_qubits = int(np.ceil(np.log2(len(complex_vector))))
    target_size = 2 ** num_qubits  # Next power of 2 (1024 for 768 complex values)
    
    # Step 3: Pad and normalize the vector
    padded_vector = pad_vector(complex_vector, target_size)
    normalized_vector = normalize(padded_vector)
    
    # Step 4: Create quantum circuit
    qc = QuantumCircuit(num_qubits)
    qubits = list(range(num_qubits))

    apply_recursive_state_preparation(qc, qubits, normalized_vector)

    # Step 5: **Explicitly save the statevector (Fix for Qiskit 1+)**
    if state_vec:
        qc.save_statevector()

    return qc

qc = prepare_amplitude_encoding(cat_vec)


In [25]:
def get_qiskit_state(vec):
    qc = prepare_amplitude_encoding(vec)
    # --- Running the circuit on AerSimulator (Statevector Mode) ---
    simulator = AerSimulator(method='statevector')
    
    # Transpile for the AerSimulator
    compiled_circuit = transpile(qc, simulator)
    
    # Run the circuit and retrieve the statevector
    job = simulator.run(compiled_circuit)
    result = job.result()
    return np.asarray(result.get_statevector())[::-1]

psi_cat = get_qiskit_state(cat_vec)
psi_dog = get_qiskit_state(dog_vec)
psi_car = get_qiskit_state(car_vec)

cat_dog_inner_prod = inner_product(psi_cat, psi_dog)
cat_dog_magnitude = np.abs(cat_dog_inner_prod)

cat_car_inner_prod = inner_product(psi_cat, psi_car)
cat_car_magnitude = np.abs(cat_car_inner_prod)

# Display the results
print("cat/dog Inner product:", cat_dog_inner_prod)
print("cat/car Inner product:", cat_car_inner_prod)

print("cat/dog Magnitude of inner product:", cat_dog_magnitude)
print("cat/car Magnitude of inner product:", cat_car_magnitude)

cat/dog Inner product: (0.8858226596196912+0.39431826432717776j)
cat/car Inner product: (0.8157520786024126+0.45544796263063625j)
cat/dog Magnitude of inner product: 0.9696229565546092
cat/car Magnitude of inner product: 0.934282773259014
